In [44]:
import requests
from bs4 import BeautifulSoup as BS

In [53]:
def get_m_content(start):
    url = "https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12-31&start={}&ref_=adv_nxt".format(start)
    print(url)
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
    page = requests.get(url, headers = headers, timeout=5)
    page.status_code
    soup = BS(page.content, 'html.parser')
    m_content = soup.findAll('div', class_ = 'lister-item-content')
    return m_content

In [54]:
import re

def parse_movie(m_content):
    m = {}

    m_header = m_content.find('h3', class_ = 'lister-item-header')
    if m_header.find('a'):
        m['title'] = m_header.find('a').get_text()
    if m_header.find('span', class_ = 'lister-item-year'):
        m['year'] = m_header.find('span', class_ = 'lister-item-year').get_text()
        if m['year']:
            m['year'] = re.findall('\d+', m['year'])

    m_muted = m_content.find('p', class_ = 'text-muted')
    if m_muted.find('span', class_ = 'certificate'):
        m['certificate'] = m_muted.find('span', class_ = 'certificate').get_text()
    if m_muted.find('span', class_ = 'runtime'):
        m['runtime'] = m_muted.find('span', class_ = 'runtime').get_text()
    if m_muted.find('span', class_ = 'genre'):
        m['genre'] = m_muted.find('span', class_ = 'genre').get_text().strip()
        m['genre'] = m['genre'].split(', ')

    m_ratings = m_content.find('div', class_ = 'ratings-bar')
    if m_ratings:
        if m_ratings.find('div', class_ = 'ratings-imdb-rating'):
            m['imdb_rating'] = float(m_ratings.find('div', class_ = 'ratings-imdb-rating').get_text().strip())
        if m_ratings.find('div', class_ = 'ratings-metascore'):
            m['metascore'] = m_ratings.find('div', class_ = 'ratings-metascore').get_text().strip()
            m['metascore'] = int(m['metascore'].split('\n')[0].strip())

    m_visible = m_content.find('p', class_ = 'sort-num_votes-visible')
    if m_visible:
        m_visible_children = m_visible.findChildren()
        if len(m_visible_children)>=2:
            m['votes'] = int(m_visible_children[1]['data-value'].replace(',',''))
            if len(m_visible_children)>=5:
                m['gross'] = int(m_visible_children[4]['data-value'].replace(',',''))

    return m

In [56]:
all_movies = []
for i in list(range(1,6600,50)):
    m_content = get_m_content(i)
    for movie in m_content:
        all_movies.append(parse_movie(movie))
        
all_movies

https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12-31&start=1&ref_=adv_nxt
https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12-31&start=51&ref_=adv_nxt
https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12-31&start=101&ref_=adv_nxt
https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12-31&start=151&ref_=adv_nxt
https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12-31&start=201&ref_=adv_nxt
https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12-31&start=251&ref_=adv_nxt
https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12-31&start=301&ref_=adv_nxt
https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12-31&start=351&ref_=adv_nxt
https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12-31&start=401&ref_=adv_nxt
https://www.imdb.com/search/title?title_type=feature&year=2019-01-01,2019-12

ReadTimeout: HTTPSConnectionPool(host='www.imdb.com', port=443): Read timed out. (read timeout=5)

In [50]:
len(all_movies)

10000

In [51]:
import pandas as pd

df = pd.DataFrame(all_movies)
df

,certificate,genre,gross,imdb_rating,metascore,runtime,title,votes,year
0,PG,"[Animation, Action, Adventure]",189901576.0,8.5,87.0,117 min,Spider-Man: Into the Spider-Verse,160484.0,[2018]
1,PG-13,"[Action, Adventure, Fantasy]",334798862.0,7.2,55.0,143 min,Aquaman,211032.0,[2018]
2,R,"[Crime, Drama, Thriller]",103804407.0,7.1,58.0,116 min,The Mule,32469.0,[2018]
3,PG-13,"[Biography, Comedy, Drama]",83956251.0,8.3,69.0,130 min,Green Book,158362.0,[2018]
4,PG-13,"[Biography, Drama, Music]",215956725.0,8.1,49.0,134 min,Bohemian Rhapsody,310806.0,[2018]
5,PG-13,"[Adventure, Family, Fantasy]",159555901.0,6.7,52.0,134 min,Fantastic Beasts: The Crimes of Grindelwald,144287.0,[2018]
6,PG-13,"[Action, Adventure, Sci-Fi]",678815482.0,8.5,68.0,149 min,Avengers: Infinity War,609381.0,[2018]
7,PG-13,"[Action, Adventure, Sci-Fi]",127195589.0,7.0,66.0,114 min,Bumblebee,65935.0,[2018]
8,R,"[Drama, Music, Romance]",215242894.0,7.8,88.0,136 min,A Star Is Born,223703.0,[2018]
9,R,"[Action, Crime, Drama]",NaN,7.4,63.0,159 min,Dragged Across Concrete,4312.0,[2018]


In [52]:
df.to_csv('imdb_2019.csv')